In [1]:
#@author = mohamed derri

#enables automatic reloading of Python modules that are imported into the notebook, 
#so that any changes made to the module are automatically reloaded without having to restart the kernel.
%reload_ext autoreload
#display Matplotlib graphs and charts directly in the notebook, rather than in a separate window.
%matplotlib inline

In [2]:
import pkg_resources
import pip
alreadyInstalledPackages = {pkg.key for pkg in pkg_resources.working_set}
requiredPackages = {"toad", "pickle"}
missing = requiredPackages - alreadyInstalledPackages
# if missing:
	# !pip install the missing ...


/usr/local/anaconda3/envs/myenv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve,auc,precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import GradientBoostingClassifier

import numpy as np
import glob
import math
import seaborn as sns 
import matplotlib.pyplot as plt
import toad
import pickle

ModuleNotFoundError: No module named 'toad'